# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle, os
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# TODO: Fill this in based on where you saved the training and testing data

training_file = "traffic_signs-data/train.p"
validation_file="traffic_signs-data/valid.p"
testing_file = "traffic_signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
X_test, y_test = shuffle(X_test, y_test)

norm_stat = np.reshape(X_train, [-1, 3]).mean(axis=0)
print(norm_stat)

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = y_train.max()+1

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

In [ ]:
# The dataset is very unbalanced. Therefore, it we need to rebalance it.
def balance_set(class_lables):
    train_class_indexes = [[] for _ in range(class_lables.max()+1)]
    n_samples = np.zeros(class_lables.max()+1)
    for i,l in enumerate(class_lables):
        train_class_indexes[l].append(i)
        n_samples[l]+=1
    
    for i, l in enumerate(train_class_indexes):
        size = len(l)
        for _ in range(int(n_samples.max())-size):
            train_class_indexes[i].append(l[np.random.randint(size)])
    
    train_class_indexes = np.reshape(np.array(train_class_indexes, dtype=np.int),[-1])
    train_class_indexes = shuffle(train_class_indexes)
    
    return train_class_indexes

trainset_balanced_ids = balance_set(y_train)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline
import cv2
import pandas as pd

def visualize_signs(signs, ids):
    n_imgs = len(signs)
    rows = int(n_imgs**0.5)
    columns = rows +1 if n_imgs-rows**2>0 else 0
    
    titles = pd.read_csv("signnames.csv")
    f = plt.figure(figsize=(18,12))
    position = 0
    for img, i in zip(signs, ids):
        position+=1
        f.add_subplot(rows, columns,position).set_title(titles.SignName[i]+"({})".format(i))
        plt.imshow(img)

visualize_signs(X_train[:20], y_train[:20])

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.
# Normalize the img data
def normalization(img_set, means):
    img_set = np.array(img_set, dtype=np.float32)
    shape = img_set.shape
    img_set = np.reshape(img_set, [-1, 3])
    img_set = np.subtract(img_set, means)
    img_set /= 255.0
    return np.reshape(img_set, shape)

In [ ]:
X_train = normalization(X_train, norm_stat)
X_test = normalization(X_test, norm_stat)
X_valid = normalization(X_valid, norm_stat)
visualize_signs(X_train[:20], y_train[:20])

### Model Architecture

In [ ]:
### Define your architecture here.
def define_model_architecture(model):
    # Reset a graph in case there
    tf.reset_default_graph()
    
    # define data input (placeholders)
    with tf.device('/gpu:0'):
        X_ph = tf.placeholder(dtype=tf.float32, shape=(None, 32,32,3), name='X')
        Y_ph = tf.placeholder(shape=(None,), name='labels', dtype=tf.uint8)
    
        net = model(X_ph, Y_ph, n_classes)
    
    return net

In [ ]:
def evaluate(net, X, Y, batch_size):
    n_samples = len(X)
    accuracy = []
    #sess = tf.get_default_session()
    pbar = tqdm(range(0, n_samples, batch_size), leave=False)
    for offset in pbar:
        start = offset
        end = start+batch_size
        pbar.set_description("Evaluate samples {}:{} out of {}".format(start,end,n_samples))
        img = X[start:end]
        labels = Y[start:end]
        a = net.eval(img, labels)
        pbar.set_postfix(accuracy="{:.1f}%".format(a*100))
        accuracy.append(a)
    pbar.close()
    return np.mean(accuracy)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
def train(trainset_balanced_ids, train_X, train_Y, validate_X, validate_Y, batch_size, n_epochs, net, path):
        
    # Initialize a saver class and variable initializer
    saver = tf.train.Saver()
    init = tf.global_variables_initializer()
    
    sess = tf.get_default_session()
    sess.run(init)
    
    #summary_op = tf.summary.merge_all()
    #summary_writer = tf.summary.FileWriter(path, sess.graph)
    
    
    
    val_acc = 0.0
    for i in range(1, n_epochs+1):
        trainset_balanced_ids = shuffle(trainset_balanced_ids)
        pbar = tqdm(range(0, len(trainset_balanced_ids), batch_size), leave=False)
        for offset in pbar:
            start = offset
            end = start+batch_size
            pbar.set_description("Epoch {}".format(i))
            batch_img = X_train[trainset_balanced_ids[start:end]]
            batch_labels = y_train[trainset_balanced_ids[start:end]]
            
            a = net.fit(batch_img, batch_labels, dropout_keep_prob=0.75)
            
            pbar.set_postfix(accuracy="{:.1f}%".format(a*100), val_accuracy="{:.1f}%".format(val_acc*100))
        ### Calculate and report the accuracy on the training and validation set.
        #pbar.close()
        val_acc = evaluate(net, validate_X, y_valid, batch_size)
        saver.save(sess, os.path.join(path, 'model.ckpt'))
        #summary_op_str = sess.run(summary_op,
        #                          feed_dict={net.X: train_X,
        #                                     net.labels: train_Y,
        #                                     net.is_training_mode: False,
        #                                     net.dropout_keep_rate: 1.0})
        #summary_writer.add_summary(summary_op_str)
    print("Model was saved to {}".format(path))
    print("Final accuracy on validation set: {:.1f}%".format(val_acc*100))
    stats = net.get_size()
    print("Final accuracy per 1k of parameters: {:.1f}%".format(val_acc / float(stats["parameters"]/10.0**5)))

In [ ]:
### Feel free to use as many code cells as needed.
# Initialize LeNet
from src.nets.LeNet import LeNet

LeNet = define_model_architecture(LeNet)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    saver = tf.train.Saver()
    #saver.restore(sess, tf.train.latest_checkpoint("logs/rebalanced_set/LeNet/"))
    
    train(trainset_balanced_ids, X_train, y_train, X_valid, y_valid, 512, 50, LeNet,"logs/rebalanced_set/LeNet/")

In [ ]:
# Initialize LeNet with batch normalization and a dropout
from src.nets.LeNet_BN_DropOut import LeNet_BN_DropOut

LeNet_BN_DropOut = define_model_architecture(LeNet_BN_DropOut)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    saver = tf.train.Saver()
    #saver.restore(sess, tf.train.latest_checkpoint("logs/LeNet_BN_DropOut/"))
    
    train(trainset_balanced_ids, X_train, y_train, X_valid, y_valid, 512, 50, LeNet_BN_DropOut,"logs/rebalanced_set/LeNet_BN_DropOut/")

In [ ]:
# Initialize a fully convolutional net with 3x3 filters with batch normalization and a dropout
from src.nets.SmallFilters import SmallFilters

SmallFilters = define_model_architecture(SmallFilters)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    train(trainset_balanced_ids, X_train, y_train, X_valid, y_valid, 512, 50, SmallFilters,"logs/rebalanced_set/SmallFilters/")

In [ ]:
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
from src.nets.SmallFilters import SmallFilters

final_net = define_model_architecture(SmallFilters)
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint("logs/rebalanced_set/SmallFilters/"))
test_accuracy = evaluate(final_net, X_test, y_test, 512)
print("SmallFilters Net yielded {:.1f}% accuracy on test set.".format(test_accuracy*100))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
keep_right_sign = plt.imread('test_samples/keep_right.jpg')
priority_road_sign = plt.imread('test_samples/priority_road.jpg')
right_of_way_at_the_next_intersection_sign = plt.imread('test_samples/right_of_way_at_the_next_intersection.jpg')
speed_limit_120_sign = plt.imread('test_samples/speed_limit_120.jpg')
yield_sign = plt.imread('test_samples/yield.jpg')

test_webimages = np.array([keep_right_sign,
                           priority_road_sign,
                           right_of_way_at_the_next_intersection_sign,
                           speed_limit_120_sign,
                           yield_sign])

test_weblabels_gt = [38,12,11,8,13]


test_webimages = np.array([cv2.resize(v, (32,32)) for v in test_webimages])

print("resulting batch shape: {}".format(test_webimages.shape))
print("Original images:")
visualize_signs(test_webimages, test_weblabels_gt)

In [ ]:
test_webimages = normalization(test_webimages, norm_stat)
print("Normalized images:")
visualize_signs(test_webimages, test_weblabels_gt)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.
titles = pd.read_csv("signnames.csv")
test_weblabels, test_weblabels_prob = final_net.infer(test_webimages)
print("Test based on web images yielded the following labels:")
for p, l in zip(test_weblabels_prob, test_weblabels):
    print("Label: {}, Probability: {:.1f}%, Title: {}".format(l, p.max()*100, titles.SignName[l]))

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
webtest_accuracy = evaluate(final_net, test_webimages, test_weblabels_gt, 5)
print("Restored SmallFiltersNet yielded {:.1f}% accuracy on a webtest set.".format(webtest_accuracy*100))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
top5_op = tf.nn.top_k(final_net.probability_op, k=5)
top5 = sess.run(top5_op, feed_dict={final_net.X: test_webimages,
                                    final_net.dropout_keep_rate: 1.0,
                                    final_net.is_training_mode: False})
for ps, inds, gt_label in zip(top5.values, top5.indices, test_weblabels_gt):
    print(titles.SignName[gt_label])
    for p,i in zip(ps, inds):
        print("Probability {:.1f}%, {} (index {})".format(p*100, titles.SignName[i], i))
    print("")

sess.close()

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 